In [ ]:
!gdown --id 1o7B8XJ33OjCo8paZytvvVsxncm9wvVke

Downloading...
From: https://drive.google.com/uc?id=1o7B8XJ33OjCo8paZytvvVsxncm9wvVke
To: /content/BigData_Homework1_Data.zip
377MB [00:05, 69.8MB/s]


In [ ]:
!unzip BigData_Homework1_Data.zip

Archive:  BigData_Homework1_Data.zip
   creating: Data/
   creating: Data/question 2_Association rules' application/
  inflating: Data/question 2_Association rules' application/browsing.txt  
   creating: Data/question 3_LSH/
   creating: Data/question 3_LSH/data/
  inflating: Data/question 3_LSH/data/patches.csv  
  inflating: Data/question 3_LSH/lsh.py  
   creating: Data/question 4_Data streams/
  inflating: Data/question 4_Data streams/counts.txt  
  inflating: Data/question 4_Data streams/counts_tiny.txt  
  inflating: Data/question 4_Data streams/hash_params.txt  
  inflating: Data/question 4_Data streams/words_stream.txt  
  inflating: Data/question 4_Data streams/words_stream_tiny.txt  


In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 74kB/s 
     |████████████████████████████████| 204kB 21.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=d2e09718805ed0165b55c147dcf6bfc2261a6c15dc4aeae23ec6872488e2caa1
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


please rename question 2 dataset folder to q2


In [ ]:
import operator
from pyspark import SparkConf, SparkContext

In [ ]:
sc = SparkContext(conf=SparkConf())

lines = sc.textFile('/content/Data/q2/browsing.txt')
baskets = lines.map(lambda l: l.split())
N = baskets.count()

baskets = baskets.map(lambda b: sorted(set(b)))

# singles

def singles_helper(basket):
    temp = []
    for item in basket:
        temp.append((item, 1))
    return temp


singles_support = baskets.flatMap(singles_helper)
singles_support = singles_support.reduceByKey(operator.add)
singles_support = singles_support.filter(lambda x: x[1] >= 100)

singles_support_b = {}
for item, support in singles_support.collect():
    singles_support_b[item] = support

singles_support_b = sc.broadcast(singles_support_b)

# doubles

def doubles_helper(basket):
    singles = singles_support_b.value
    temp = []
    for i in range(len(basket)):
        if basket[i] in singles:
            for j in range(i):
                if basket[j] in singles:
                    temp.append(((basket[j], basket[i]), 1)) # basket is sorted
    return temp


doubles_support = baskets.flatMap(doubles_helper)
doubles_support = doubles_support.reduceByKey(operator.add)
doubles_support = doubles_support.filter(lambda x: x[1] >= 100)

def confidence_doubles_helper(double_support):
    double, support = double_support
    support = float(support)
    u, v = double
    singles = singles_support_b.value
    uv_conf = support / singles[u]
    vu_conf = support / singles[v]
    return (('%s -> %s' % (u, v), uv_conf),
            ('%s -> %s' % (v, u), vu_conf))


doubles_conf = doubles_support.flatMap(confidence_doubles_helper)
doubles_conf = doubles_conf.sortBy(lambda x: (-x[1], x[0]))

doubles_support_b = {}
for entry, support in doubles_support.collect():
    doubles_support_b[entry] = support

doubles_support_b = sc.broadcast(doubles_support_b)

# triples

def triples_helper(basket):
    doubles = doubles_support_b.value
    singles = singles_support_b.value
    temp = []
    for i in range(len(basket)):
        if basket[i] not in singles:
            continue
        for j in range(i):
            if basket[j] not in singles:
                continue
            if (basket[j], basket[i]) not in doubles:
                continue
            for k in range(j):
                if basket[k] not in singles:
                    continue
                if (basket[k], basket[j]) not in doubles:
                    continue
                if (basket[k], basket[i]) not in doubles:
                    continue
                temp.append(((basket[k], basket[j], basket[i]), 1))
    return temp


triples_support = baskets.flatMap(triples_helper)
triples_support = triples_support.reduceByKey(operator.add)
triples_support = triples_support.filter(lambda x: x[1] >= 100)

def confidence_triples_helper(triple_support):
    doubles = doubles_support_b.value
    triple, support = triple_support
    support = float(support)
    u, v, w = triple
    uv_w = support / doubles[u, v]
    uw_v = support / doubles[u, w]
    vw_u = support / doubles[v, w]
    return (('(%s, %s) -> %s' % (u, v, w), uv_w),
            ('(%s, %s) -> %s' % (u, w, v), uw_v),
            ('(%s, %s) -> %s' % (v, w, u), vw_u))


triples_conf = triples_support.flatMap(confidence_triples_helper)
triples_conf = triples_conf.sortBy(lambda x: (-x[1], x[0]))

print(str(doubles_conf.take(5)))
print('\n')
print(str(triples_conf.take(5)))


[('DAI93865 -> FRO40251', 1.0), ('GRO85051 -> FRO40251', 0.999176276771005), ('GRO38636 -> FRO40251', 0.9906542056074766), ('ELE12951 -> FRO40251', 0.9905660377358491), ('DAI88079 -> FRO40251', 0.9867256637168141)]


[('(DAI23334, ELE92920) -> DAI62779', 1.0), ('(DAI31081, GRO85051) -> FRO40251', 1.0), ('(DAI55911, GRO85051) -> FRO40251', 1.0), ('(DAI62779, DAI88079) -> FRO40251', 1.0), ('(DAI75645, GRO85051) -> FRO40251', 1.0)]
